#MA707 Report - Estimator Pipelines (spring 2019, DataHeroes)

## Introduction
This notebook contains code to create estimator pipelines which include the feature selection classes and the wrapped estimator classes. All the different train-test dataset defined in the `Preprocessing pipeline` will be fitted and tested to get their individual score at the default parameters as defined. 
The code will be reused in the `Investigation` notebook where it will be fitted to various pipelines with different hyperparameters in GridSeach and get the `mean_test_scores` and their rank. 

The train-test dataset used for fitting the models and predicting in this notebooks are:
- [Dataframe 1: `fea_tgt_coal_tfidf_pdf` ](https://bentley.cloud.databricks.com/#notebook/1364451/command/1603805)
- [Dataframe 2: `fea_tgt_coal_cnt_pdf`](https://bentley.cloud.databricks.com/#notebook/1364451/command/1603807)
- [Dataframe 3: `fea_tgt_ore_tfidf_pdf`](https://bentley.cloud.databricks.com/#notebook/1364451/command/1603808)
- [Dataframe 4: `fea_tgt_ore_cnt_pdf`](https://bentley.cloud.databricks.com/#notebook/1364451/command/1603809)

## Contents
1. Setup

## 1. Setup

In [5]:
%run "./2. Preprocessing pipelines"

### Lasso Pipelines

Least Absolute Shrinkage Selector Operator in short Lasso regression not only helps in reducing over-fitting but it can help in feature selection. It has the ability to shrink the estimated coefficient for the model to exactly zero, reducing the number of features and serve as a feature selection tool as well as the regression tool at the same time. By adding a biasing term to the regression optimization function to reduce the effect of collinearity. Lasso helps reduce the multicollinearity of endogenous variables in models which creates inaccurate estimates of the regression coefficients. 

The [EstimatorLasso()](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607095) wrapper class have the default parameters `alpha=1`  and `normalize=True` so the regressors X will be normalized before regression. 
 
**Note:** In order to get the best value of `alpha`, the data in the training set will be used to get estimate coefficients’ value for every value of `alpha`. In the validation set, different values of the product of `alpha` and coefficient estimates will be assessed. The one which has lower error value will get selected. This selected value of coefficients’ will again be assessed by test data set.

In [8]:
%python 
def get_lasso_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.linear_model import LinearRegression, Ridge
  from sklearn.feature_selection import chi2
  return Pipeline(steps=[
    ('lso', EstimatorLasso())
  ])

Fit the different train-test datasets into the above estimator pipeline and compare the scores.

In [10]:
get_lasso_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 210 ]: 0.6996321665031258

In [11]:
get_lasso_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 211 ]: 0.7221271878743873

In [12]:
get_lasso_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 212 ]: 0.7180596111631561

In [13]:
get_lasso_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 213 ]: 0.6869873953834693

The Lasso regression is performed on the 3 day lag data of `bci` and 3 day lag data of counts and frequency of `tags`, `title` and `content` of the different datasets created in the pre-processing noteboook. The table below shows the results for each dataset. <table>
    <tr>
        <td>`bci_coal_pdf`</td><td>`bci_ironore_pdf`</td><td>`CountVectorizer`</td><td>`TfidfVectorizer`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td>Yes</td><td></td><td>72.21%</td>
    </tr>
    <tr>
        <td></td><td>Yes</td><td>Yes</td><td></td><td>68.69%</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td></td><td>Yes</td><td>69.96</td>
    </tr>
    <tr>
        <td></td><td>Yes</td><td></td><td>Yes</td><td>71.80%</td>
    </tr>  
</table>


From the results, it can be seen that the train-test dataset created by performing the feature extraction `CountVectorizer` method on the `bci_coal_pdf` and `TfidfVectorizer` on the `bci_ironore_pdf` both have the R Square (R2) value close to 72%.  R-square is the coefficient of determination which determines the proportion of the variability explained by the model. Later in the investigation notebook, these two datasets will try to run `GridSearchCV` to fine tune the hyper parameters and improve the model efficiency.

##PCA Lasso Pipeline

Extending the Lasso model would be to add another feature reduction process to reduce the features and in process to check the model prediction.  

Add the [FeatureSelectionPCA](https://bentley.cloud.databricks.com/#notebook/1607074/command/1607080) wrapper class which has the base class of PCA (Principal component analysis). It is a technique for feature extraction — so it combines the input variables in a specific way, then can drop the “least important” variables while still retaining the most valuable parts of all of the variables. So here below creating a pipeline by using the `FeatureSelectionPCA()` wrapper class which have default parameters
 - `n_components=10` :- Means here it will create 10 new featuers using the existing features.

In [17]:
%python 
def get_pca_lasso_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.linear_model import LinearRegression, Ridge
  return Pipeline(steps=[
    ('pca', FeatureSelectionPCA()),
    ('lso', EstimatorLasso())
  ])

Fit the estimator pipeline with the two classes and compare the scores with the different training and testing sets.

In [19]:
get_pca_lasso_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 215 ]: 0.5705490311724806

In [20]:
get_pca_lasso_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 216 ]: 0.5531753188892263

In [21]:
get_pca_lasso_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 217 ]: 0.5721601812600043

In [22]:
get_pca_lasso_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 218 ]: 0.5521581673391068

%md PCA-Lasso pipeline performed on the below combination of datasets, the table below shows the results.  
 <table>
    <tr>
        <td>`bci_coal_pdf`</td><td>`bci_ironore_pdf`</td><td>`CountVectorizer`</td><td>`TfidfVectorizer`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td>Yes</td><td></td><td>55.31%</td>
    </tr>
    <tr>
        <td></td><td>Yes</td><td>Yes</td><td></td><td>55.21%</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td></td><td>Yes</td><td>57.05%</td>
    </tr>
    <tr>
        <td></td><td>Yes</td><td></td><td>Yes</td><td>57.21%</td>
    </tr>  
</table>

As we see `bci_coal_pdf with CountVectorizer` and `bci_ironore_pdf with TfidfVectorizer` both have the R Square (R2) value of around 55% to 57%. Eventhough there is a drop in the variability with the addition of PCA, but as default only 10 features have been selected as default. This estimator pipeline will be tried  using `GridSearchCV` with addtion of many features by fine tuning the hyper parameters to check any improvement in the model efficiency.

### ElasticNet Pipelines

ElasticNet is a third commonly used model of regression which incorporates penalties from both Ridge and Lasso models. Ridge regression model is explained in the following sections. Addition to setting and choosing a lambda value elastic net also allows to tune the alpha parameter where 𝞪 = 0 corresponds to ridge and 𝞪 = 1 to lasso. Therefore an alpha value can be chosen between 0 and 1 to optimize the elastic net. ElasticNet keeps the group effect in the case of highly correlated variables, rather than zeroing some of them like Lasso and also no limitation on the number of selected variables. 

The [EstimatorElasticNet()](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607099) wrapper class which have default parameters `alpha=0.5`it gives the equal weight to both Ridge and Lasso lambda values and `normalize=True` which regressors X will be normalized before regression. 
 
 **Note:** In order to get the best value of `alpha`, the data in the training set will be used to get estimate coefficients’ value for every value of `alpha`. In the validation set, different values of the product of `alpha` and coefficient estimates will be assessed. The one which has lower error value will get selected. This selected value of coefficients’ will again be assessed by test data set.

ElasticNet regression is performed on the below combination of datasets.

In [27]:
%python 
def get_elasticnet_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.linear_model import LinearRegression, Ridge
  return Pipeline(steps=[
    ('ela', EstimatorElasticNet())
  ])

In [28]:
get_elasticnet_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 220 ]: -1.6026201763340326

In [29]:
get_elasticnet_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 221 ]: -1.595733877104839

In [30]:
get_elasticnet_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 222 ]: -1.5649748609984355

In [31]:
get_elasticnet_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 223 ]: -1.5663365684287252

d %md The table below shows the results (r-square value) for the different datasets fitted into the Elaastic net estimator pipeline.
  <table>
    <tr>
        <td>`bci_coal_pdf`</td><td>`bci_ironore_pdf`</td><td>`CountVectorizer`</td><td>`TfidfVectorizer`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td>Yes</td><td></td><td> -1.595</td>
    </tr>
    <tr>
        <td></td><td>Yes</td><td>Yes</td><td></td><td>-1.566</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td></td><td>Yes</td><td>-1.603</td>
    </tr>
    <tr>
        <td></td><td>Yes</td><td></td><td>Yes</td><td>-1.565</td>
    </tr>  
</table>

As we see for all the combinations have the R Square (R2) the proportion of the variability explained by the models is negative which means the model is not good. Will try to run `GridSearchCV` to check if any scope of improvement, also like to check the same by adding PCA too. Since all of the test scores are negative, we try Elastic Net model with PCA to improve the model performance

Extending the Elastic model by adding another feature reduction process PCA to check if the overall predicton will improve.

In [33]:
%python 
def get_pca_elasticnet_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.linear_model import LinearRegression, Ridge
  return Pipeline(steps=[
    ('pca', FeatureSelectionPCA()),
    ('ela', EstimatorElasticNet())
  ])

In [34]:
get_pca_elasticnet_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 225 ]: -1.652031389105387

In [35]:
get_pca_elasticnet_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 226 ]: -1.6512141723974794

In [36]:
get_pca_elasticnet_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 227 ]: -1.641098898459107

In [37]:
get_pca_elasticnet_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 228 ]: -1.6402270398874297

ElasticNet regression with PCA performed on the above combination of datasets. As noticed ealier the  R Square of the models is still negative which means the model is not good.

### SVR Pipelines

Support Vector Regression (SVR) uses the same principles as the SVM for classification, with only a few minor differences. In simple regression we try to minimise the error rate. While in SVR we try to fit the error within a certain threshold.  Objective with SVR is to basically consider the points that are within the boundary line, so best fit line is the line hyperplane that has maximum number of points.

Build the `get_svr_pipeline` function which returns a pipeline [Estimatorsvr](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607103) which calls the base class `SVR` with default parameters `kernel='rbf', gamma='auto', C=100`. Call the function and fit the pipeline with
4 different datasets then get the R score on the relative test datasets. Compare the scores.

In [42]:
%python 
def get_svr_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.svm               import SVR
  from sklearn.feature_selection import chi2
  return Pipeline(steps=[
    ('svr', EstimatorSVR())
  ])

In [43]:
get_svr_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 230 ]: -2.8959249004785232

In [44]:
get_svr_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 231 ]: -2.89630577106593

In [45]:
get_svr_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 232 ]: -2.8811050016842903

In [46]:
get_svr_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 233 ]: -2.8810047965738446

The scores are as following: -2.896, -2.896, -2.881, -2.881. The scores are all 
negative for the default parameters in the model, the model appears to be not working very well with the datasets. SVR regression also results in a R Square (R2) value which is negative so the model is not good to be used. So inorder to get better results from this model, it will be tested using the GridSearchCV with different parameters other than the default values during investigation.

###Ridge Pipelines

Ridge regression is a regression technique for continuous value prediction. It is a simple techniques to reduce model complexity and prevent over-fitting which may result from simple linear regression. Using a ridge regression method helps reduce the multicollinearity of endogenous variables in models. Multicollinearity creates inaccurate estimates of the regression coefficients.

Generally when overfitting happens, the regression coefficients’ values becomes very huge. Ridge regression is used to quantify the overfitting of the data through measuring the magnitude of coefficients.  The parameter `alpha` as defined in the wrapper class `EstimatorRidge` in *Notebook 0.4* is the tuning parameter to balance the fit of data and magnitude of coefficients.

The class [EstimatorRidge()](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607091) which calls on the base class `Ridge` from sklearn.linear_model package, with all its default parameter `normalize=False` and `solver='auto'` which chooses the solver automatically based on the type of data.

**Note:** Inorder to get the best value of `alpha`, the data in the training set will be used to get estimate coefficients’ value for every value of `alpha`. In the validation set, different values of the product of `alpha` and coefficient estimates will be assessed. The one which has lower error value will get selected. This selected value of coefficients’ will again be assessed by test data set.

Build `get_ridge_pipeline` function, it returns a pipeline with the estimator class [estimatorRidge](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607091). Call the function and fit the pipeline with
4 different datasets then get the R score on the relative test datasets.

In [51]:
def get_ridge_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.linear_model import LinearRegression, Ridge
  return Pipeline(steps=[
    ('rdg', EstimatorRidge())
  ])

In [52]:
get_ridge_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 236 ]: -0.971180299717989

In [53]:
get_ridge_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 237 ]: -0.8353216169750179

In [54]:
get_ridge_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 238 ]: -0.7446500148891255

In [55]:
get_ridge_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 239 ]: -0.5491851344825782

The scores are as following: -0.971, -0.835, -0.744, -0.549. The scores are all 
negative with the default parameters for the wrapper class. Hence in the Investigation notebook different hyperparameters will be tested to fine tune the model during gridsearch and results will be compared and ranked based on their `mean_test_score` value.

### Decision Tree

A decision tree is a largely used non-parametric effective machine learning modeling technique for regression and classification problems. To find solutions a decision tree makes sequential, hierarchical decision about the outcomes variable based on the predictor data.

Hierarchical means the model is defined by a series of questions that lead to a class label or a value when applied to any observation. Once set up the model acts like a protocol in a series of “if this occurs then this occurs” conditions that produce a specific result from the input data.

A Non-parametric method means that there are no underlying assumptions about the distribution of the errors or the data. It basically means that the model is constructed based on the observed data.

Build the `get_DecisionTree_pipeline` function which returns a pipeline with an estimator [EstimatorDecisiontree](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607111) which has a base class of `DecisionTreeRegressor` from `sklearn.tree`. The default parameter for the estimator is set as `max_depth=5, max_leaf_nodes=10`. Call the function and fit the pipeline with
4 different datasets then get the R score on the relative test datasets.

In [60]:
def get_DecisionTree_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.tree     import DecisionTreeRegressor
  return Pipeline(steps=[
    ('dtr',EstimatorDecisiontree())
  ])

In [61]:
get_DecisionTree_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 241 ]: 0.12304391486498313

In [62]:
get_DecisionTree_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 242 ]: 0.24612626349563116

In [63]:
get_DecisionTree_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 243 ]: 0.08139281914497176

In [64]:
get_DecisionTree_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 244 ]: 0.10022421639724532

The scores are as following: 0.123, 0.246, 0.081, 0.100. It can be concluded that
the decision tree works best with `bci_coal_pdf` dataset containing features extracted using the CountVectorizer method.

### Random Forest

Random forests, also known as random decision forests, are a popular ensemble method that can be used to build predictive models for both classification and regression problems. Ensemble methods use multiple learning models to gain better predictive results — in the case of a random forest, the model creates an entire forest of random uncorrelated decision trees to arrive at the best possible answer.

In [68]:
def get_RandomForest_pipeline():
  from sklearn.pipeline import FeatureUnion, Pipeline
  from sklearn.tree     import DecisionTreeRegressor
  return Pipeline(steps=[
    ('rf',EstimatorRandomForest())
  ])

Build `get_RandomForest_pipeline` function which returns a pipeline with the estimator [EstimatorRandomForest](https://bentley.cloud.databricks.com/#notebook/1607089/command/1607114), which has the base class `RandomForestRegressor` with the default parameters `n_estimators=5`, and `max_leaf_nodes=10`. Call the function and fit the pipeline with
4 different datasets then get the R score on the relative test datasets.

In [70]:
get_RandomForest_pipeline() \
  .fit  (trn_coal_tfidf_fea_pdf, trn_coal_tfidf_tgt_ser) \
  .score(tst_coal_tfidf_fea_pdf, tst_coal_tfidf_tgt_ser)

Out[ 246 ]: 0.566372283449996

In [71]:
get_RandomForest_pipeline() \
  .fit  (trn_coal_cnt_fea_pdf, trn_coal_cnt_tgt_ser) \
  .score(tst_coal_cnt_fea_pdf, tst_coal_cnt_tgt_ser)

Out[ 247 ]: 0.6397947673873587

In [72]:
get_RandomForest_pipeline() \
  .fit  (trn_ore_tfidf_fea_pdf, trn_ore_tfidf_tgt_ser) \
  .score(tst_ore_tfidf_fea_pdf, tst_ore_tfidf_tgt_ser)

Out[ 248 ]: 0.6528499130954133

In [73]:
get_RandomForest_pipeline() \
  .fit  (trn_ore_cnt_fea_pdf, trn_ore_cnt_tgt_ser) \
  .score(tst_ore_cnt_fea_pdf, tst_ore_cnt_tgt_ser)

Out[ 249 ]: 0.5787878651636065

The scores are as following: 0.566, 0.639, 0.653, 0.579. It can be concluded that the decision tree works best with the dataset created from the merged `bci_ironore_pdf` features and `tfidfVectorizer` features.

## Summary

In this notebook different estimator pipelines were built and tested with their default parameters on the four different datasets created in Preprocessing pipeline.
Here's the score rank of all 7 models with the best outcome:

- Lasso Regression: 0.722 
- Random Forest: 0.691
- Lasso with PCA: 0.572
- Decision Tree: 0.246
- Ridge Regression: -0.547
- Elastic Net: - 1.602
- SVR: -2.881

The best Pipeline with a high `mean_test_score` of the R-square values is **Lasso Regression** with default parameters of `alpha=1` and `normalize='True'`. The scores of the model fitted and predicted on the train test dataset created from `bci_coal_pdf` dataset with features created using the `CounVectorizer` method and the `bci_ironore_pdf` dataset with additional features created using the `TfidfVectorizer` method are as below. <table>
    <tr>
        <td>`bci_coal_pdf`</td><td>`bci_ironore_pdf`</td><td>`CountVectorizer`</td><td>`TfidfVectorizer`</td><td>`R Square`</td>
    </tr>
    <tr>
        <td>Yes</td><td></td><td>Yes</td><td></td><td>72.21%</td>
    </tr>
    <tr>
    <tr>
        <td></td><td>Yes</td><td></td><td>Yes</td><td>71.80%</td>
    </tr>  
</table>
Hence in the Investigation models where gridsearch will be performed with different hyperparameters, these two combination of train-test datasets will be used to train and model and get the best score of each model and then select the best model with its hyperparameters.